In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()


from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import accuracy_score
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
# from keras.models import load_model

from imblearn.over_sampling import SMOTE
import pickle

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

from sklearn.preprocessing import PolynomialFeatures
from imblearn.over_sampling import SMOTE
import pickle
from lightgbm.sklearn import LGBMClassifier

## you'll have to pip install LGBM
# !pip install lightgbm

/usr/local/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
data_train = pd.read_csv("../lipika/cleaned_2013_14", low_memory = False);
data_test = pd.read_csv("../lipika/cleaned_2015", low_memory = False);

In [3]:
data_test = data_test.dropna()

In [4]:
data_test.columns == data_train.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [15]:
def split_data(df, cols):
    x = df.drop(cols, axis = 1)
    y = df.paid
    return x, y

cols_to_drop_training = ['loan_status', 'paid', 'amnt', 'total_pymnt', 'term_adj', 'zip_code']
x_train_initial, y_train_initial = split_data(data_train, cols_to_drop_training)
x_test, y_test = split_data(data_test, cols_to_drop_training)

In [16]:
x_train_initial.shape

(351757, 44)

In [17]:
# sm = SMOTE(random_state=1, ratio = 1.0)
# x_train, y_train = sm.fit_sample(x_train_initial, y_train_initial)

In [18]:
# x_test = x_test.dropna()
# y_test = y_test.dropna()

In [19]:
# with open('Tuned_RF.pkl', 'rb') as file:  
#     rf = pickle.load(file)

# with open('QDA.pkl', 'rb') as file:  
#     qda = pickle.load(file)

with open('Tuned_LGBM.pkl', 'rb') as file:  
    lgbm = pickle.load(file)

# with open('../anthony/AdaboostGS.pkl', 'rb') as file:  
#     adaboost = pickle.load(file, encoding='latin1')

# with open('../bhaven/Tuned_logReg_all_training_data.pkl', 'rb') as file:  
#     logreg = pickle.load(file, encoding='latin1')

In [20]:
# models = [rf, qda, lgbm, adaboost, logreg]
models = [lgbm]

In [21]:
def create_df(data_1, data_2, penal = 0.5):
    df = pd.DataFrame(data_1)
    
    df['int_rate'] = data_2['int_rate']
    df['amnt'] = data_2['amnt']
    df['total_pymnt'] = data_2['total_pymnt']
    df['term_adj'] = data_2['term_adj']
    df['ROI'] = (((1 + df['int_rate'])*(1-df['proba']))+(df['proba']*drr*penal))-1

    df['Real_ROI'] = df['amnt']*(((df['total_pymnt']/df['amnt'])**(1/df['term_adj']))-1)
    df['annualized_amnt'] = df['amnt']*(1/df['term_adj'])
    
    return df

In [22]:
distress = data_train[data_train.paid == 1]
drr = (np.sum(distress.total_pymnt)/np.sum(distress.amnt))**(1/np.mean(distress.term))
print(drr)

0.8999757837386156


In [25]:
train_data = {}
test_data= {}
df_train = {}
df_test = {}

for model in models:
    print(model)
    tr = {'proba' : model.predict_proba(x_train_initial)[:, 1], 
                         'pred' : model.predict(x_train_initial), 
                         'true_val' : y_train_initial}
    df_tr = create_df(tr, data_train)
    train_data[model] = tr
    df_train[model] = df_tr
    
    print(x_test.shape)
    te = {'proba' : model.predict_proba(x_test)[:, 1], 
                         'pred' : model.predict(x_test), 
                         'true_val' : y_test}    
    print(len(te['proba']))
    df_te = create_df(te, data_test)    
    test_data[model] = te
    df_test[model] = df_te

Pipeline(memory=None,
     steps=[('POLY', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)), ('LGBM', LGBMClassifier(bagging_fraction=0.7, boosting_type='gbdt', class_weight=None,
        colsample_bytree=1.0, importance_type='split', lambda_l2=0,
        learning_rate=0.1, max_depth=15, metric='binar...0, reg_lambda=0.0, silent=True,
        subsample=1.0, subsample_for_bin=200000, subsample_freq=0))])
(346843, 44)
346843


In [26]:
# #load neural net
# nn = load_model('NN_final_model.h5')

# tr = {'proba' : model.predict_proba(x_train)[:, 1], 
#                          'pred' : model.predict(x_train), 
#                          'true_val' : y_train}

# nn_train = {'proba': nn.predict(x_train)[:, 1], 'true_val': y_train}
# nn_test = {'proba': nn.predict(x_test)[:, 1], 'true_val': y_test}

# nn_train['pred'] = np.where(nn_train['proba'] > 0.5, 1, 0)
# nn_test['pred'] = np.where(nn_test['proba'] > 0.5, 1, 0)

# df_train['nn'] = create_df(nn_train, data_train)
# df_test['nn'] =  create_df(nn_test, data_test)


In [27]:
# checking = df_train.copy()
# x = checking.keys()[0]
# y = checking.keys()[1]
# checking['lgbm'] = checking.pop(x)
# checking['qda'] = checking.pop(y)

In [28]:
# (checking['lgbm']).to_csv('lgbm_preds_training')
# (checking['qda']).to_csv('qda_preds_training')

In [29]:
# checking2 = df_test.copy()
# x = checking2.keys()[0]
# y = checking2.keys()[1]
# checking2['lgbm'] = checking2.pop(x)
# checking2['qda'] = checking2.pop(y)

In [30]:
# (checking2['lgbm']).to_csv('lgbm_preds_test')
# (checking2['qda']).to_csv('qda_preds_test')

In [31]:
# df = pd.read_csv('qda_preds_training')

In [32]:
# df.describe()

In [33]:
with open('lgbm_train_no_smote.pickle', 'wb') as handle:
    pickle.dump(df_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('lgbm_test_no_smote.pickle', 'wb') as handle:
    pickle.dump(df_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open('filename.pickle', 'rb') as handle:
#     b = pickle.load(handle)



In [ ]:
# # train_data[model] = {'proba' : model.predict_proba(x_train)[:, 1], 'pred' : model.predict(x_train), 'true_val' : y_train}
# # test_data = {'proba' : rf_complex.predict_proba(x_test)[:, 1], 'pred' : rf_complex.predict(x_test),  'true_val' : y_test}

# df_train = create_df(train_data, data_train)
# df_test =  create_df(test_data, data_test)

In [ ]:
# baseline_train = np.mean(df_train.ROI)
# baseline_test = np.mean(df_test.ROI)
# print(baseline_train, baseline_test)
# alpha_low = 0
# alpha_high = -np.inf

In [ ]:
# print(roc_auc_score(df_train.true_val, df_train.pred))
# print(roc_auc_score(df_test.true_val, df_test.pred))

In [ ]:
# print(np.sum(df_train.true_val)/df_train.shape[0])
# print(1 - np.sum(df_test.true_val)/df_test.shape[0])

In [ ]:
# loans_to_buy_train = df_train[(df_train.ROI > alpha_low*baseline_train) & (df_train.ROI < alpha_high*baseline_train)]
# loans_to_buy_test = df_test[(df_test.ROI > alpha_low*baseline_test) & (df_test.ROI < alpha_high*baseline_test)]

In [ ]:
# def return_performance(df_sub, df_main):
#     perf_subset = np.sum(df_sub.Real_ROI)/np.sum(df_sub.annualized_amnt)
#     perf_mainset = np.sum(df_main.Real_ROI)/np.sum(df_main.annualized_amnt)
#     return (perf_subset, perf_mainset, perf_subset - perf_mainset)

In [ ]:
# training_set_performance = return_performance(loans_to_buy_train, df_train)
# testing_set_performance = return_performance(loans_to_buy_test, df_test)

In [ ]:
# fig = plt.figure(figsize=(15,6));
# ax = plt.subplot(1,1,1);

# ax.hist(df_test.loc[df_test.true_val == 0,'proba'], density = True, bins = 50, label='Paid Off', alpha=0.6);
# ax.hist(df_test.loc[df_test.true_val == 1,'proba'], density = True, bins = 50, label='Defaulted', alpha=0.5);
# ax.set_title('Initial Predicted ROI for Defaulted & Non-Defaulted Customers with Higher Order Random Forest')
# ax.legend(loc='best')

# plt.show()

In [ ]:
# fig = plt.figure(figsize=(15,6));
# ax = plt.subplot(1,1,1);

# ax.hist(df_train.loc[df_train.true_val == 0,'proba'], bins = 50, label='Paid Off', alpha=0.6);
# ax.hist(df_train.loc[df_train.true_val == 1,'proba'], bins = 50, label='Defaulted', alpha=0.5);
# ax.set_title('Initial Predicted ROI for Defaulted & Non-Defaulted Customers with Higher Order Random Forest')
# ax.legend(loc='best')

# plt.show()

In [ ]:
# sns.kdeplot(df_train.loc[df_train.true_val == 0,'proba'], label = 'Paid Out');
# sns.kdeplot(df_train.loc[df_train.true_val == 1,'proba'], label = 'Defaulted');

In [ ]:
# sns.set(rc={'figure.figsize':(16,8)})
# sns.kdeplot(df_test.loc[df_test.true_val == 0,'proba'], label = 'Paid Out');
# sns.kdeplot(df_test.loc[df_test.true_val == 1,'proba'], label = 'Defaulted', linestyle='--');
# plt.ylim(0, 3)
# plt.xlim(0, 1);
# plt.xlabel('Probability of Default', size = 15);
# plt.ylabel('Density of Distributions', size = 15);
# plt.legend(prop={'size': 13});
# plt.savefig('default_probability_distribution.png', bbox_inches='tight')

In [ ]:
# sns.kdeplot(df_test['ROI'], label = '2015 Data');
# sns.kdeplot(df_train['ROI'], label = '2014 Data', linestyle='--');
# plt.ylim(0, 5)
# plt.xlim(-0.5, 0.1);
# plt.xlabel('Predicted Return on Investment', size = 15);
# plt.ylabel('Density of Distributions', size = 15);
# plt.legend(prop={'size': 13});
# plt.savefig('ROI_distribution.png', bbox_inches='tight')